<a href="https://colab.research.google.com/github/ArijaK/QuestionAnswering/blob/main/QA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Flags.
# Data preprocessing.
DATASET = 'squad_v2'
MODEL_CHECKPOINT = 'deepset/roberta-base-squad2'
# Maximum length of a feature (question and context).
MAX_LENGTH = 512
# Number of overlapping tokens.
STRIDE = 128

## Data preparation

In [2]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
# Easy way to load the dataset.
from datasets import load_dataset
dataset = load_dataset(DATASET)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [4]:
# Preprocessing.
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [6]:
# Check if a fast tokenizer is implemented.
tokenizer.is_fast
# Call the tokenizer on a simple example.
inputs = tokenizer('Where can I buy cat food?', 'Cat food is sold in all pet stores. You can buy cat food online too.')
tokenizer.decode(inputs['input_ids'])
# TODO: Add some additional simple examples for better understanding of the preprocessing.

'<s>Where can I buy cat food?</s></s>Cat food is sold in all pet stores. You can buy cat food online too.</s>'

In [9]:
def preprocess_data(data):
  data['question'] = [q.strip() for q in data['question']]

  inputs = tokenizer(
      data['question'],
      data['context'],
      truncation='only_second',
      max_length=MAX_LENGTH,
      stride=STRIDE,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding='max_length',
  )

  sample_mapping = inputs.pop('overflow_to_sample_mapping')
  offset_mapping = inputs.pop('offset_mapping')

  inputs['start_positions'] = []
  inputs['end_positions'] = []

  for i, offsets in enumerate(offset_mapping):
    input_ids = inputs['input_ids'][i]
    cls_index = input_ids.index(tokenizer.cls_token_id)

    sequence_ids = inputs.sequence_ids(i)
    sample_index = sample_mapping[i]
    answers = data['answers'][sample_index]
    if len(answers['answer_start']) == 0:
      inputs['start_positions'].append(cls_index)
      inputs['end_positions'].append(cls_index)
    else:
      start_char = answers['answer_start'][0]
      end_char = start_char + len(answers['text'][0])

      # Find the start and end of the context.
      token_start_index = 0
      while sequence_ids[token_start_index] != 1:
        token_start_index += 1

      token_end_index = len(input_ids) - 1
      while sequence_ids[token_end_index] != 1:
        token_end_index -= 1

      if not(offsets[token_start_index][0] <= start_char and
             offsets[token_end_index][1] >= end_char):
        inputs['start_positions'].append(cls_index)
        inputs['end_positions'].append(cls_index)
      else:
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
          token_start_index += 1
        inputs['start_positions'].append(token_start_index - 1)
        while offsets[token_end_index][1] >= end_char:
          token_end_index -= 1
        inputs['end_positions'].append(token_end_index + 1)

  return inputs

In [12]:
tokenized_dataset = dataset['train'].map(
    preprocess_data,
    batched=True,
    remove_columns=dataset['train'].column_names,
)
len(dataset['train']), len(tokenized_dataset)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

(130319, 130550)